# Evalutation of regression and classification models : model assessement

This code is a complement to PyTerK environment notebooks. It allows to gather all desired model evaluation in a table, using similar method than PyTerK. 

In this notebook, we are evaluating different models on the same dataset. The train data, test data and models are loaded for each iteration and k-folds. Predictions are performed from train data. Extected and predicted values are stored.

Regression models are evaluatied through adjusted $R^2$ of the regression between expected and predicted values. For adjusted $R^2$, the size of the dataset correspond to ```len(y_pred)```and the number of independant variables is 1 as predicted values are supposed to be equal to expected values. 
__________________________________________________________________________________________________________________________________

*Example of tree structure obtained from training of 3 Neural Network (NN), Random Forest (RF) and SVM regression models with different hyperparameters, for different ouputs.*

* campaign08 
    * NN_r : regression NN models
        * Fit_E : output of the model
            * Fit_E_0000 : model n°1 *eg: 50x50 layers*
            * Fit_E_0001 : model n°2 *eg: 100x100 layers*
            * Fit_E_0002 : model n°3 *eg: 50x100 layers*
        * Fit_H
        * Fit_CI
        * Fit_IQ 
    * NN_c
    * RF_r
    * RF_c
    * SVM_r
    * SVM_c



## Import libraries 

In [1]:
import numpy as np
import os, json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import hamming_loss

## Regression model

### Get paths for models to evaluate: stored results of iterative k-fold crossvalidation trainings (```run```) for different outputs (```fit```)

In [2]:
run_dir=os.getenv('RUN_DIR')
directory=f"{run_dir}campaign08/"
runs=['SVM_r','NN_r','RF_r']
fit=['Fit_E','Fit_H', 'Fit_CI','Fit_IQ']

### Calculation of adjusted $R^2$ for each model and output

In [3]:
# For each run/kind of models and each outputs/predictions
for num_model in range (0,len(runs)):
    for num_pred in range (0,len(fit)):
        # Let's go through the different kind of models and outputs — eg: NN_r/Fit_E
        run_dir=directory+runs[num_model]+'/'+fit[num_pred]+'/'

        # Subdirs= all models of a kind, with different hyperparemeters — eg : NN_r/Fit_E/Fit_E_0000
        subdirs = [f.path for f in os.scandir(run_dir) if f.is_dir() and not f.name.startswith('.')]
        subdirs = sorted(subdirs)
        
        # Dataframe to store the metrics of each training : initialisation with an empty column
        df_r2_adj_models=pd.DataFrame(columns=['model0']) 
        # Dataframe to store the mean and std of metrics for each model: : initialisation with an empty column
        df_r2_adj_mean_std=pd.DataFrame(columns=['model0'],index=['R2 adj mean','R2 adj std'])
        
        
        for s in subdirs:
            r2_adj_table=[]
            about= json.load(open(s+'/about.json'))
            name_model=about['args']['model_id']
            # For all iteration and all k-fold
            iteration=[f.path for f in os.scandir(s) if f.is_dir() and not f.name.startswith('.')]
            for it in iteration:
                k_fold=[f.path for f in os.scandir(it) if f.is_dir() and not f.name.startswith('.')]
                for k in k_fold:
                    with open(k+'/yytest.json') as fd:
                        yy = json.load(fd)
                        y_pred=np.reshape(np.array(yy['y_pred']),(np.shape(np.array(yy['y_pred']))[0]))
                        y_test=np.reshape(np.array(yy['y_test']),(np.shape(np.array(yy['y_test']))[0]))

                        # Compute R2 and R2 adjusted 
                        corr_matrix = np.corrcoef(y_test,y_pred)
                        corr = corr_matrix[0,1]
                        R2 = corr**2
                        R2_adj=1-((1-R2)*(len(y_test)-1))/(len(y_test)-1-1)
                        r2_adj_table.append(R2_adj)
            
            df_r2_adj_models[name_model]=r2_adj_table
            df_r2_adj_mean_std[name_model]=[np.mean(r2_adj_table),np.std(r2_adj_table)]                                              
            

        # Print table of evaluation of each model (mean and std of metrics)
        display(runs[num_model]+'_'+fit[num_pred])
        del df_r2_adj_mean_std['model0'] # delete initial columns
        df_r2_adj_mean_std=df_r2_adj_mean_std.T
        display(df_r2_adj_mean_std)

        # Gives the best model.
        maxR2adj=df_r2_adj_mean_std['R2 adj mean'].max()
        bestmodel=df_r2_adj_mean_std['R2 adj mean'].idxmax()
        print('The best model is '+bestmodel +' with R2_adj='+str(maxR2adj)+' +/- '+str(df_r2_adj_mean_std['R2 adj std'][bestmodel])+'\n')
        
        # Save evaluation of each model. 
        df_r2_adj_mean_std.to_csv(f'{directory}results/regression/R2_adj_mean_std_'+runs[num_model]+'_'+fit[num_pred]+'.csv')                                                    

'SVM_r_Fit_E'

,R2 adj mean,R2 adj std
sklearn-SVR-poly2-0.1-1,0.647699,0.024147
sklearn-SVR-poly2-5-1,0.651727,0.023628
sklearn-SVR-poly2-10-1,0.658680,0.022977
sklearn-SVR-poly2-0.1-100,0.682201,0.028609
sklearn-SVR-poly2-5-100,0.686498,0.026017
...,...,...
sklearn-nuSVR-rbf-0.5-100,0.947646,0.008901
sklearn-nuSVR-rbf-0.8-100,0.947123,0.009600
sklearn-nuSVR-rbf-0.2-1000,0.953695,0.008978
sklearn-nuSVR-rbf-0.5-1000,0.956642,0.009867


The best model is sklearn-nuSVR-rbf-0.5-1000 with R2_adj=0.9566417362370863 +/- 0.00986651277808897



/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


'SVM_r_Fit_H'

...

,R2 adj mean,R2 adj std
sklearn-rfr-50-5,0.899151,0.054253
sklearn-rfr-100-5,0.903485,0.054572
sklearn-rfr-150-5,0.904040,0.045646
sklearn-rfr-50-10,0.889739,0.059868
sklearn-rfr-100-10,0.893803,0.055620
sklearn-rfr-150-10,0.892405,0.061392
sklearn-rfr-50-20,0.827926,0.068570
sklearn-rfr-100-20,0.830497,0.069056
sklearn-rfr-150-20,0.832244,0.066572


The best model is sklearn-rfr-150-5 with R2_adj=0.9040397665018973 +/- 0.04564580446183668



'RF_r_Fit_IQ'

,R2 adj mean,R2 adj std
sklearn-rfr-50-5,0.913056,0.038119
sklearn-rfr-100-5,0.907810,0.048386
sklearn-rfr-150-5,0.909711,0.047444
sklearn-rfr-50-10,0.895356,0.049818
sklearn-rfr-100-10,0.896033,0.058031
sklearn-rfr-150-10,0.897222,0.051899
sklearn-rfr-50-20,0.791604,0.083171
sklearn-rfr-100-20,0.792785,0.090748
sklearn-rfr-150-20,0.792939,0.078343


The best model is sklearn-rfr-50-5 with R2_adj=0.9130560511311426 +/- 0.038119206534769925



## Classification models

### Get paths for models to evaluate: stored results of iterative k-fold crossvalidation trainings (```run```) for different outputs (```classifications```)
Add prediction type : to compute the metrics we need only classes. If the prediction type is other, then we correct it.

In [7]:
run_dir=os.getenv('RUN_DIR')
directory=f"{run_dir}campaign08/"
runs=['NN_class','RF_class','SVM_class']
classification=['Class_CI','Class_DRX']
predict_type_list=['sigmoid','classes','classes']

### Calculation of F1-score, accuracy, Hamming loss, confusion matrix for each model and output

In [33]:
# For each run/kind of models and each outputs/predictions
for num_model in range (0,len(runs)):
    for num_pred in range (0,len(classification)):

        # Let's go through the different kind of models and outputs — eg: RF_c/Class_CI
        run_dir=directory+runs[num_model]+'/'+classification[num_pred]+'/'
        

        # Subdirs= all models of a kind, with different hyperparemeters — eg : RF_c/Class_CI/Class_CI_0000
        subdirs = [f.path for f in os.scandir(run_dir) if f.is_dir() and not f.name.startswith('.')]
        subdirs = sorted(subdirs)
        

        # Dataframe to store the metrics of each training : initialisation with an empty column
        df_metrics_models=pd.DataFrame(columns=['model0'])

        # Dataframe to store the mean and std of metrics for each model: : initialisation with an empty column
        df_metrics_mean_std=pd.DataFrame(columns=['model0'],
                                         index=['accuracy mean','accuracy std',
                                                'recall amorphous mean', 'recall amorphous std', 
                                                'precision amorphous mean','precision amorphous std',
                                                'F1 amorphous mean','F1 amorphous std',
                                                'recall cristalline mean', 'recall cristalline std', 
                                                'precision cristalline mean','precision cristalline std',
                                                'F1 cristalline mean','F1 cristalline std',
                                                'F1 weighted mean' , 'F1 weighted std',
                                                'Hamming loss mean', 'Hamming loss std'])
        
        
        i=0
        for s in subdirs:
            accuracy_table=[]
            recall_table_amorphe=[]
            precision_table_amorphe=[]
            F1_table_amorphe=[]
            recall_table_crist=[]
            precision_table_crist=[]
            F1_table_crist=[]
            F1_table_weighted=[]
            Hml_table=[]
            y_pred_list=[]
            y_test_list=[]
            
            
            about= json.load(open(s+'/about.json'))
            name_model=about['args']['model_id']
            
            # For all iteration and all k-fold
            iteration=[f.path for f in os.scandir(s) if f.is_dir() and not f.name.startswith('.')]
            for it in iteration:
                k_fold=[f.path for f in os.scandir(it) if f.is_dir() and not f.name.startswith('.')]
                for k in k_fold:
                    with open(k+'/yytest.json') as fd:
                        yy = json.load(fd)
                        y_pred=np.reshape(np.array(yy['y_pred']),(np.shape(np.array(yy['y_pred']))[0]))
                        
                        y_test=np.reshape(np.array(yy['y_test']),(np.shape(np.array(yy['y_test']))[0]))
                        
                        # Transform output into class
                        predict_type=predict_type_list[num_model]
                        if predict_type=='softmax':
                            y_pred = np.array( [ np.argmax(y) for y in y_pred] )

                        if predict_type=='sigmoid':
                            y_pred = np.array( [ 0 if y<0.5 else 1 for y in y_pred] )

                        if predict_type=='classes':
                            y_pred = y_pred.squeeze()


                        metrics =pd.DataFrame(classification_report(y_test,y_pred,output_dict=True)).T
                        #display(metrics)
                        #print(metrics['recall'].keys)
                        amorph_key=metrics['recall'].keys()[0]
                        crist_key=metrics['recall'].keys()[1]
                        
                        accuracy_table.append(metrics['precision']['accuracy'])
                        recall_table_amorphe.append(metrics['recall'][amorph_key])
                        precision_table_amorphe.append(metrics['precision'][amorph_key])
                        F1_table_amorphe.append(metrics['f1-score'][amorph_key])
                        recall_table_crist.append(metrics['recall'][crist_key])
                        precision_table_crist.append(metrics['precision'][crist_key])
                        F1_table_crist.append(metrics['f1-score'][crist_key])
                        F1_table_weighted.append(metrics['f1-score']['weighted avg'])
                        Hml_table.append(hamming_loss(y_test, y_pred))
                        y_pred_list.append(y_pred)
                        y_test_list.append(y_test)
                        
            y_pred_list=np.concatenate(y_pred_list[:])  
            y_test_list=np.concatenate(y_test_list[:])             
            
            df_metrics_mean_std[name_model]=[np.mean(accuracy_table),np.std(accuracy_table),
                                             np.mean(recall_table_amorphe),np.std(recall_table_amorphe),
                                             np.mean(precision_table_amorphe),np.std(precision_table_amorphe),
                                             np.mean(F1_table_amorphe),np.std(F1_table_amorphe),
                                             np.mean(recall_table_crist),np.std(recall_table_crist),
                                             np.mean(precision_table_crist),np.std(precision_table_crist),
                                             np.mean(F1_table_crist),np.std(F1_table_crist),
                                             np.mean(F1_table_weighted),np.std(F1_table_weighted),
                                             np.mean(Hml_table),np.std(Hml_table)]                                            
                                             
            cm = confusion_matrix( y_test_list, y_pred_list, normalize="pred")
            disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['amorphous','cristalline'])
            disp.plot()
            plt.title(name_model)
            plt.show()    

        
        display(runs[num_model]+'_'+classification[num_pred])
        del df_metrics_mean_std['model0']
        
        # Print table of evaluation of each model (mean and std of metrics)
        df_metrics_mean_std=df_metrics_mean_std.T
        display(df_metrics_mean_std)

        # Gives the best model base on F1 weighted score and check that it coherent with accuracy and Hamming loss
        max_F1_w=df_metrics_mean_std['F1 weighted mean'].max()
        max_F1_w=df_metrics_mean_std['F1 weighted mean'].max()
        bestmodel=df_metrics_mean_std['F1 weighted mean'].idxmax()
        accuracy_best=(df_metrics_mean_std['accuracy mean'].idxmax()==bestmodel)
        hml_best=(df_metrics_mean_std['Hamming loss mean'].idxmin()==bestmodel)
        print('The best model for phase classification is '+bestmodel +' with F1w='+str(max_F1_w)+' +/- '+str(df_metrics_mean_std['F1 weighted std'][bestmodel])+'\n')
        print('The best accuracy is obtained for the same model :'+str(accuracy_best)+'\n')
        print('The lowest Hamming loss is obtained for the same model :'+str(hml_best)+'\n')

        # Save metrics mean and std 
        #df_metrics_mean_std.to_csv(f'{directory}results/Raw_R2_'+runs[num_model]+'_'+fit[num_pred]+'.csv')
        df_metrics_mean_std.to_csv(f'{directory}results/class/metrics_mean_std_'+runs[num_model]+'_'+classification[num_pred]+'.csv')                                                    

./Model_assessment_choice/run/campaign08_bis/NN_class/Class_CI/


'NN_class_Class_CI'

,accuracy mean,accuracy std,recall amorphous mean,recall amorphous std,precision amorphous mean,precision amorphous std,F1 amorphous mean,F1 amorphous std,recall cristalline mean,recall cristalline std,precision cristalline mean,precision cristalline std,F1 cristalline mean,F1 cristalline std,F1 weighted mean,F1 weighted std,Hamming loss mean,Hamming loss std
keras-50x50,0.952653,0.022787,0.947445,0.028827,0.955695,0.026699,0.951317,0.023577,0.957157,0.027159,0.949690,0.027772,0.953185,0.023459,0.952640,0.022798,0.047347,0.022787
keras-100x100,0.952447,0.025494,0.945149,0.032286,0.958057,0.028396,0.951288,0.026129,0.959051,0.029440,0.947204,0.030555,0.952845,0.026097,0.952424,0.025536,0.047553,0.025494
keras-50x50x50,0.953240,0.023119,0.946364,0.029943,0.958397,0.026683,0.952065,0.023389,0.959468,0.027414,0.948181,0.028975,0.953556,0.024004,0.953225,0.023136,0.046760,0.023119
keras-100x100x100,0.955330,0.026545,0.946341,0.031470,0.962916,0.032816,0.954213,0.026886,0.963688,0.033765,0.948537,0.030542,0.955734,0.027433,0.955308,0.026583,0.044670,0.026545
keras-50x50x50x50,0.956307,0.022894,0.946564,0.028872,0.964049,0.028530,0.954968,0.024145,0.965329,0.027662,0.948987,0.027046,0.956854,0.023001,0.956292,0.022905,0.043693,0.022894
keras-100x100x100x100,0.956244,0.021325,0.945668,0.028811,0.964940,0.026712,0.954887,0.021782,0.966017,0.027281,0.948462,0.027307,0.956874,0.021891,0.956221,0.021339,0.043756,0.021325
keras-100x100x50x50,0.955957,0.022732,0.941617,0.034296,0.967641,0.025055,0.954096,0.024214,0.969066,0.024735,0.945803,0.030311,0.957005,0.022462,0.955914,0.022769,0.044043,0.022732
keras-50x100x100x50,0.956310,0.022860,0.944244,0.031398,0.965977,0.029470,0.954593,0.024117,0.967318,0.028969,0.947911,0.028454,0.957186,0.023011,0.956280,0.022857,0.043690,0.022860
keras-50x50x100x100,0.957543,0.023782,0.946024,0.031849,0.966285,0.027903,0.955778,0.025497,0.967814,0.026588,0.949770,0.029211,0.958459,0.023472,0.957517,0.023797,0.042457,0.023782


The best model for phase classification is keras-50x50x100x100 with F1w=0.957517089149146 +/- 0.02379734128075483

The best accuracy is obtained for the same model :True

The lowest Hamming loss is obtained for the same model :True

./Model_assessment_choice/run/campaign08_bis/NN_class/Class_DRX/


'NN_class_Class_DRX'

...

The best model for phase classification is sklearn-SVC-rbf-100 with F1w=0.9434072849883937 +/- 0.023513499375759982

The best accuracy is obtained for the same model :True

The lowest Hamming loss is obtained for the same model :True

./Model_assessment_choice/run/campaign08_bis/SVM_class/Class_DRX/


/Users/elisegarel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'SVM_class_Class_DRX'

,accuracy mean,accuracy std,recall amorphous mean,recall amorphous std,precision amorphous mean,precision amorphous std,F1 amorphous mean,F1 amorphous std,recall cristalline mean,recall cristalline std,precision cristalline mean,precision cristalline std,F1 cristalline mean,F1 cristalline std,F1 weighted mean,F1 weighted std,Hamming loss mean,Hamming loss std
sklearn-SVC-poly2-1,0.751687,0.041123,0.732975,0.083620,0.607788,0.077849,0.659606,0.057001,0.763451,0.057527,0.851626,0.049676,0.803060,0.036575,0.756558,0.040094,0.248313,0.041123
sklearn-SVC-poly2-100,0.789591,0.038910,0.771436,0.072090,0.658431,0.081431,0.706146,0.056962,0.799868,0.056557,0.875689,0.041670,0.834368,0.034848,0.793373,0.037576,0.210409,0.038910
sklearn-SVC-poly2-1000,0.789675,0.039276,0.774582,0.074940,0.656512,0.069718,0.707319,0.054440,0.798472,0.047615,0.876085,0.047479,0.834201,0.035351,0.793502,0.038606,0.210325,0.039276
sklearn-SVC-poly3-1,0.809000,0.051895,0.608468,0.202731,0.818440,0.123192,0.662569,0.113492,0.921678,0.067140,0.827397,0.090065,0.865545,0.034862,0.796334,0.066120,0.191000,0.051895
sklearn-SVC-poly3-100,0.863527,0.033129,0.846475,0.074485,0.773212,0.082970,0.802905,0.047883,0.874168,0.052634,0.919694,0.040241,0.894645,0.027652,0.865033,0.032350,0.136473,0.033129
sklearn-SVC-poly3-1000,0.864996,0.031864,0.867569,0.063342,0.763027,0.077733,0.808161,0.048303,0.864733,0.048705,0.929700,0.035335,0.894699,0.026746,0.867112,0.030670,0.135004,0.031864
sklearn-SVC-rbf-1,0.925987,0.026081,0.918454,0.043020,0.866799,0.060796,0.890354,0.039933,0.930044,0.032405,0.958105,0.022902,0.943462,0.020738,0.926619,0.025561,0.074013,0.026081
sklearn-SVC-rbf-100,0.938727,0.021558,0.928182,0.038728,0.892360,0.053797,0.908603,0.032995,0.944077,0.028318,0.963540,0.019697,0.953375,0.016949,0.939098,0.021233,0.061273,0.021558
sklearn-SVC-rbf-1000,0.935745,0.022993,0.925557,0.047339,0.885092,0.057528,0.903261,0.038236,0.941275,0.027704,0.962084,0.024548,0.951161,0.017651,0.936172,0.022722,0.064255,0.022993
sklearn-SVC-sigmoid-1,0.509186,0.063486,0.250758,0.105260,0.252018,0.088280,0.247861,0.091414,0.639238,0.066910,0.632054,0.068753,0.633398,0.056894,0.506488,0.067210,0.490814,0.063486


The best model for phase classification is sklearn-SVC-rbf-100 with F1w=0.9390977249753454 +/- 0.021232722770738902

The best accuracy is obtained for the same model :True

The lowest Hamming loss is obtained for the same model :True

